# Second Merger

Merge between df_unmerged_companies.csv and new aida's extraction

In [4]:
import pandas as pd
import numpy as np
import re
from thefuzz import process, fuzz
from src.ingestion import download_from_drive
from tqdm.notebook import tqdm

In [ ]:
# PAY ATTENTION: if not updated, it will overwrite the files

# set starting db links
df_crunchbase = pd.read_csv('data/df_unmerged_companies_2.csv')
df_aida = download_from_drive("bolai_acquired")

# set names of the files where the results will be saved
df_merged_companies = "df_merged_companies_3.csv"
df_unmerged_companies = "df_unmerged_companies_3.csv"


Error: [Errno 2] No such file or directory: '../data/file_id.json'
Please ensure the required file '../data/file_id.json' exists.


In [ ]:
# Load the databases
df_anagrafica = df_aida
#df_anagrafica = download_from_drive("csv_startup_anagrafica")
#df_crunchbase = download_from_drive("estrazione_crunchbase_5k")

# Display the first few rows of each dataframe
print("Startup Anagrafica Dataset:")
display(df_anagrafica.head())

print("\nCrunchbase Dataset:")
display(df_crunchbase.head())

In [ ]:
def pulisci_stringa(s: str) -> str:
    """
    Enhanced string cleaning function with more sophisticated preprocessing
    for better company name matching.
    """
    if not isinstance(s, str):
        if s is None:
            return ""
        else:
            return str(s)
    
    s = s.strip().lower()
    
    # Handle common company type abbreviations (more comprehensive list)
    s = re.sub(r'\b(srl|s\.r\.l|s\.r\.l\.|s.r.l|spa|s\.p\.a|s\.p\.a\.|s.p.a|' + 
               r'inc|incorporated|llc|ltd|limited|gmbh|' + 
               r'corp|corporation|co\.|company|group|holding|' + 
               r'sas|sapa|snc|scarl)(?=\s|$|\b)', '', s)
    
    # Handle common abbreviations
    s = s.replace("int'l", "international")
    s = s.replace("tech.", "technology")
    s = s.replace("tech", "technology")
    s = s.replace("&", "and")
    
    # Remove geographical designations that might differ
    s = re.sub(r'\b(italy|italia|italian|italiano|italiana)\b', '', s)
    
    # Remove special characters and extra spaces
    s = re.sub(r'[^\w\s]', ' ', s)
    
    # Remove common filler words
    s = re.sub(r'\b(the|di|del|della|e|a|in|for|of)\b', ' ', s)
    
    # Replace multiple spaces with single space and trim
    s = re.sub(r'\s+', ' ', s).strip()
    
    return s

def calculate_composite_score(name1, name2):
    """
    Calculate a composite score using multiple string matching algorithms
    to better identify company name matches.
    
    Returns: float between 0-100 representing match confidence
    """
    # Standard ratios check different aspects of string similarity
    ratio = fuzz.ratio(name1, name2)
    partial_ratio = fuzz.partial_ratio(name1, name2)
    token_sort_ratio = fuzz.token_sort_ratio(name1, name2)
    token_set_ratio = fuzz.token_set_ratio(name1, name2)
    
    # Calculate a weighted average - giving more weight to token methods
    # as they handle word order differences better for company names
    weighted_score = (
        ratio * 0.15 +
        partial_ratio * 0.25 +
        token_sort_ratio * 0.3 +
        token_set_ratio * 0.3
    )
    
    # Additional bonus for exact token matching
    tokens1 = set(name1.split())
    tokens2 = set(name2.split())
    
    # If there are significant shared tokens, boost the score
    if len(tokens1) > 0 and len(tokens2) > 0:
        shared = tokens1.intersection(tokens2)
        if len(shared) >= 2:  # At least 2 significant words match
            weighted_score += min(10, len(shared) * 2.5)  # Bonus up to 10 points
    
    # Cap at 100
    return min(100, weighted_score)

def enhanced_fuzzy_merge(
    df1: pd.DataFrame,
    df2: pd.DataFrame,
    key1: str,
    key2: str,
    threshold: int = 75
) -> pd.DataFrame:
    """
    Enhanced version of fuzzy_merge that uses multiple matching techniques
    for more accurate company name matching.
    """
    # Copia dei DataFrame per non modificare gli originali
    df1_clean = df1.copy()
    df2_clean = df2.copy()
    
    # Creiamo colonne "pulite" per il matching con la funzione migliorata
    df1_clean['key_clean'] = df1_clean[key1].apply(pulisci_stringa)
    df2_clean['key_clean'] = df2_clean[key2].apply(pulisci_stringa)
    
    # Create clean keys and make a dictionary for lookup
    unique_keys2 = df2_clean['key_clean'].unique().tolist()
    
    # Lista per salvare i risultati
    risultati = []
    
    # Track already matched names in df2 to avoid duplicates
    matched_df2_indices = set()

    for idx1, row in tqdm(df1_clean.iterrows(), total=len(df1_clean), desc="Matching records"):
        name_to_match = row['key_clean']
        
        # Skip empty names
        if not name_to_match.strip():
            continue
            
        # Get top 5 potential matches instead of just one
        top_matches = process.extract(name_to_match, unique_keys2, 
                                     scorer=fuzz.token_set_ratio, limit=5)
        
        best_match = None
        best_score = 0
        best_match_row = None
        
        # Evaluate each potential match with our composite score
        for match_name, base_score in top_matches:
            # Calculate more sophisticated score
            composite_score = calculate_composite_score(name_to_match, match_name)
            
            # Find matching rows
            match_rows = df2_clean[df2_clean['key_clean'] == match_name]
            
            # Skip if we've already used all these matches
            if all(idx in matched_df2_indices for idx in match_rows.index):
                continue
                
            # Get first available match
            for idx2, match_row in match_rows.iterrows():
                if idx2 not in matched_df2_indices and composite_score > best_score:
                    best_score = composite_score
                    best_match = match_name
                    best_match_row = match_row
                    best_match_idx = idx2
                    break
        
        # Process the best match if it meets the threshold
        if best_match and best_score >= threshold:
            # Mark this df2 row as matched
            matched_df2_indices.add(best_match_idx)
            
            # Build combined data dictionary
            combined_data = {}
            
            # Data from df1
            for col in df1_clean.columns:
                if col != 'key_clean':  # Exclude auxiliary column
                    combined_data[f"anagrafica_{col}"] = row[col]
            
            # Data from df2
            for col in df2_clean.columns:
                if col != 'key_clean':  # Exclude auxiliary column
                    combined_data[f"crunchbase_{col}"] = best_match_row[col]
            
            combined_data['similarity_score'] = best_score
            combined_data['composite_match'] = True
            
            risultati.append(combined_data)
        else:
            # If no good match, keep just df1 data
            combined_data = {}
            for col in df1_clean.columns:
                if col != 'key_clean':
                    combined_data[f"anagrafica_{col}"] = row[col]
            
            for col in df2_clean.columns:
                if col != 'key_clean':
                    combined_data[f"crunchbase_{col}"] = None
                    
            combined_data['similarity_score'] = best_score if best_score else 0
            combined_data['composite_match'] = False
            risultati.append(combined_data)
    
    # Add remaining df2 rows that weren't matched
    if len(matched_df2_indices) < len(df2_clean):
        for idx2, row in df2_clean.iterrows():
            if idx2 not in matched_df2_indices:
                combined_data = {}
                
                # Empty df1 data
                for col in df1_clean.columns:
                    if col != 'key_clean':
                        combined_data[f"anagrafica_{col}"] = None
                
                # df2 data
                for col in df2_clean.columns:
                    if col != 'key_clean':
                        combined_data[f"crunchbase_{col}"] = row[col]
                
                combined_data['similarity_score'] = 0
                combined_data['composite_match'] = False
                risultati.append(combined_data)
    
    # Convert to DataFrame
    df_risultati = pd.DataFrame(risultati)
    return df_risultati

In [ ]:
# Ensure ipywidgets is installed to fix the IProgress error
#%pip install ipywidgets

# Eseguiamo il merge con una soglia appropriata
df_merged = enhanced_fuzzy_merge(df_anagrafica, df_crunchbase, "Ragione sociale", "Organization Name", threshold=85)

# Visualizziamo un campione dei risultati
print(f"Numero di record totali dopo il merge: {len(df_merged)}")
df_merged.head()

In [ ]:
# Creiamo un ID unico per ogni coppia abbinata
# Utilizziamo una combinazione dei valori originali per creare un ID significativo

def create_unique_id(row):
    # Prendiamo il nome dalla colonna anagrafica se disponibile, altrimenti da crunchbase
    company_name = str(row['anagrafica_Ragione sociale']) if pd.notna(row['anagrafica_Ragione sociale']) else \
                  str(row['crunchbase_Organization Name']) if pd.notna(row['crunchbase_Organization Name']) else 'unknown'
    
    # Pulizia base per l'ID
    clean_name = re.sub(r'[^\w]', '', company_name.lower())
    
    # Aggiungiamo un valore numerico progressivo per garantire l'unicità
    return f"comp_{clean_name[:20]}"

# Creiamo inizialmente l'ID senza controllo duplicati
df_merged['company_id'] = df_merged.apply(create_unique_id, axis=1)

# Controlliamo e gestiamo i duplicati aggiungendo un numero progressivo
id_counts = df_merged['company_id'].value_counts()
duplicated_ids = id_counts[id_counts > 1].index.tolist()

# Per ogni ID duplicato, aggiungiamo un contatore
for dup_id in duplicated_ids:
    # Identifichiamo tutte le righe con questo ID
    mask = df_merged['company_id'] == dup_id
    # Aggiungiamo un contatore progressivo
    df_merged.loc[mask, 'company_id'] = [
        f"{dup_id}_{i}" for i in range(1, mask.sum() + 1)
    ]

# Verifichiamo che non ci siano più duplicati
print(f"ID unici: {df_merged['company_id'].nunique()}")
print(f"Totale record: {len(df_merged)}")

# Visualizziamo i risultati con i nuovi ID
df_merged[['company_id', 'anagrafica_Ragione sociale', 'crunchbase_Organization Name', 'similarity_score']].head(10)

In [ ]:
# Gestione delle colonne - manteniamo i nomi originali ma riorganizziamo il dataset
# Creiamo un nuovo DataFrame più pulito con le colonne principali

cols_to_keep = [
    'company_id',  # Il nostro ID unico
    'anagrafica_Ragione sociale',
    'crunchbase_Organization Name',
    'similarity_score'
    # Aggiungi qui altre colonne di interesse
]

df_final = df_merged[cols_to_keep].copy()

# Rinominiamo le colonne per chiarezza mantenendo i nomi originali
df_final = df_final.rename(columns={
    'anagrafica_Ragione sociale': 'Ragione sociale',
    'crunchbase_Organization Name': 'Organization Name'
})

# Visualizza il dataset finale
df_final.head()

In [ ]:
# Opzionale: salva il dataset finale
# df_final.to_csv('merged_company_data.csv', index=False)

# Statistiche finali
print(f"Totale record nel dataset finale: {len(df_final)}")
print(f"Record con match di alta qualità (score >= 90): {(df_final['similarity_score'] >= 90).sum()}")
print(f"Record con match medio (score 80-90): {((df_final['similarity_score'] >= 80) & (df_final['similarity_score'] < 90)).sum()}")
print(f"Record con match basso (score < 80): {(df_final['similarity_score'] < 80).sum() if 'similarity_score' in df_final.columns else 'N/A'}")

In [ ]:
df_final.tail()

In [ ]:
# Conteggio delle corrispondenze tra anagrafica_Ragione sociale e crunchbase_Organization Name

# Filtriamo le righe dove entrambi i campi non sono null
matched_companies = df_merged[
    pd.notna(df_merged['anagrafica_Ragione sociale']) & 
    pd.notna(df_merged['crunchbase_Organization Name'])
]

# Contiamo quante corrispondenze abbiamo trovato
total_matches = len(matched_companies)
total_anagrafica = df_merged['anagrafica_Ragione sociale'].notna().sum()
total_crunchbase = df_merged['crunchbase_Organization Name'].notna().sum()

print(f"Totale aziende con corrispondenza tra i due database: {total_matches}")
print(f"Totale aziende nel database anagrafica: {total_anagrafica}")
print(f"Totale aziende nel database crunchbase: {total_crunchbase}")
print(f"Percentuale di aziende anagrafica con match: {(total_matches / total_anagrafica * 100):.2f}%")
print(f"Percentuale di aziende crunchbase con match: {(total_matches / total_crunchbase * 100):.2f}%")

# Visualizziamo alcuni esempi di corrispondenze con i relativi score
print("\nEsempi di corrispondenze trovate (ordinate per score):")
display(matched_companies[['company_id', 'anagrafica_Ragione sociale', 
                          'crunchbase_Organization Name', 'similarity_score']]
       .sort_values(by='similarity_score', ascending=False).head(10))

# Distribuzione degli score per le corrispondenze trovate
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.histplot(matched_companies['similarity_score'], bins=20)
plt.title('Distribuzione degli score per le corrispondenze trovate')
plt.xlabel('Score di similarità')
plt.ylabel('Numero di corrispondenze')
plt.axvline(x=85, color='red', linestyle='--', label='Threshold')
plt.legend()
plt.show()

In [ ]:
# Save the df_merged DataFrame to a CSV file in the 'data' folder
df_merged.to_csv(f"data/{df_merged_companies}", index=False)

# Unmerged companies

In [ ]:
# Filter rows where "crunchbase_Organization Name" is not null and "anagrafica_Ragione sociale" is null
df_crunchbase_unmerged = df_merged[
    df_merged['crunchbase_Organization Name'].notna() & df_merged['anagrafica_Ragione sociale'].isna()
]

# Select only the columns that begin with "crunchbase_"
df_crunchbase_unmerged = df_crunchbase_unmerged[
    [col for col in df_merged.columns if col.startswith("crunchbase_")]
]

# Rename the columns to remove the "crunchbase_" prefix
df_crunchbase_unmerged.columns = df_crunchbase_unmerged.columns.str.replace('crunchbase_', '', regex=False)
df_crunchbase_unmerged.head()

# Display the resulting DataFrame
df_crunchbase_unmerged.shape

# Save the crunchbase_only_companies DataFrame to a CSV file in the 'data' folder
df_crunchbase_unmerged.to_csv(f"data/{df_unmerged_companies}", index=False)

In [ ]:
def extract_unmatched_crunchbase_locations():
    """
    Estrae gli Headquarters Location dal database Crunchbase per le aziende
    che sono presenti solo in Crunchbase (senza corrispondenza in anagrafica).
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame contenente Organization Name e Headquarters Location
        delle aziende che sono solo in Crunchbase
    """
    # Filtriamo le righe dove c'è un valore in crunchbase_Organization Name
    # ma non in anagrafica_Ragione sociale
    unmatched_cb = df_merged[
        pd.notna(df_merged['crunchbase_Organization Name']) & 
        pd.isna(df_merged['anagrafica_Ragione sociale'])
    ].copy()
    
    # Selezioniamo solo le colonne di interesse
    result = unmatched_cb[['crunchbase_Organization Name', 'crunchbase_Headquarters Location']]
    
    # Rinominiamo le colonne per maggiore chiarezza
    result = result.rename(columns={
        'crunchbase_Organization Name': 'Organization Name',
        'crunchbase_Headquarters Location': 'Headquarters Location'
    })
    
    # Riordiniamo e resettiamo l'indice
    result = result.sort_values(by='Organization Name').reset_index(drop=True)
    
    return result

# Eseguiamo la funzione e mostriamo i risultati
unmatched_locations = extract_unmatched_crunchbase_locations()

# Statistiche sulle sedi non corrispondenti
print(f"Totale organizzazioni solo in Crunchbase (senza match in anagrafica): {len(unmatched_locations)}")

# Contiamo la distribuzione delle sedi (top 10)
if 'Headquarters Location' in unmatched_locations.columns and not unmatched_locations.empty:
    location_counts = unmatched_locations['Headquarters Location'].value_counts().head(10)
    print("\nTop 10 sedi delle organizzazioni non corrispondenti:")
    display(location_counts)
    
    # Percentuale di valori mancanti nella colonna Headquarters Location
    missing_pct = unmatched_locations['Headquarters Location'].isna().mean() * 100
    print(f"\nPercentuale di valori mancanti nella colonna 'Headquarters Location': {missing_pct:.2f}%")

# Mostriamo i primi risultati
print("\nEsempi di organizzazioni solo in Crunchbase:")
display(unmatched_locations.head(10))

In [ ]:
def get_crunchbase_only_companies():
    """
    Estrae i nomi delle aziende che sono presenti solo nel database Crunchbase
    e non hanno corrispondenza nel database Anagrafica.
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame contenente solo i nomi delle aziende presenti esclusivamente in Crunchbase
    """
    # Filtriamo le righe dove c'Ã¨ un valore in crunchbase_Organization Name
    # ma non in anagrafica_Ragione sociale
    crunchbase_only = df_merged[
        pd.notna(df_merged['crunchbase_Organization Name']) & 
        pd.isna(df_merged['anagrafica_Ragione sociale'])
    ].copy()
    
    # Selezioniamo solo la colonna con i nomi delle organizzazioni
    result = crunchbase_only[['crunchbase_Organization Name']].copy()
    
    # Rinominiamo la colonna per maggiore chiarezza
    result = result.rename(columns={
        'crunchbase_Organization Name': 'Organization Name'
    })
    
    # Rimuoviamo eventuali duplicati
    result = result.drop_duplicates().reset_index(drop=True)
    
    # Riordiniamo alfabeticamente
    result = result.sort_values(by='Organization Name').reset_index(drop=True)
    
    return result

# Eseguiamo la funzione e mostriamo i risultati
crunchbase_only_companies = get_crunchbase_only_companies()

# Statistiche sulle aziende
print(f"Totale aziende presenti solo in Crunchbase (senza match in anagrafica): {len(crunchbase_only_companies)}")

# Mostriamo i primi risultati
print("\nEsempi di aziende solo in Crunchbase:")
display(crunchbase_only_companies.head(15))

# Opzionale: verifichiamo la coda della lista
print("\nUltime aziende della lista:")
display(crunchbase_only_companies.tail(5))

In [ ]:
# Save the crunchbase_only_companies DataFrame to a CSV file in the 'data' folder
crunchbase_only_companies.to_csv(f"data/{df_unmerged_companies}", index=False)